## Modules

In [103]:
import os
from tqdm import tqdm

from PIL import Image
import numpy as np
from scipy.interpolate import interp1d
from scipy.signal import find_peaks, correlate, butter, freqs, filtfilt
from scipy.ndimage import gaussian_filter, median_filter, shift
from scipy.stats import sem, zscore
from scipy.optimize import curve_fit

import tifffile as tff
import seaborn as sns
import seaborn_image as isns
import cmcrameri.cm as cmc
from matplotlib import pyplot as plt
from skimage.registration import phase_cross_correlation
# import ants
import csv
from matplotlib import colormaps as clm

# cmap = 'cmc.batlow'
# sns.set_palette(cmap)

## Open data

In [113]:
pathBase = r"Z:\gGermain\2024-09-17\3"

HbO = zscore(np.loadtxt(pathBase + "\\HbO_4rois.csv", skiprows=1, delimiter=',', usecols=(1, 2, 3, 4)), axis=0)
# print(len(HbO))
HbR = zscore(np.loadtxt(pathBase + "\\HbR_4rois.csv", skiprows=1, delimiter=',', usecols=(1, 2, 3, 4)), axis=0)
# print(len(HbR))
HbT = zscore(np.loadtxt(pathBase + "\\HbT_4rois.csv", skiprows=1, delimiter=',', usecols=(1, 2, 3, 4)), axis=0)
# print(len(HbT))
gcamp = zscore(np.loadtxt(pathBase + "\\470_4rois.csv", skiprows=1, delimiter=',', usecols=(1, 2, 3, 4)), axis=0)[:len(HbT), :]
# print(len(gcamp))


# gcamp = np.loadtxt(pathBase + "\\470.csv", delimiter=',', usecols=(1), skiprows=1)
# Hb = np.load(pathBase + "\\computedHb.npy")
# (HbO, HbR, HbT) = Hb

ts = np.load(pathBase + "\\470ts.npy")[:len(HbT)]

In [114]:
# air puffs

AP_times = np.array([  12.01,   35.2 ,   46.51,   74.12,   91.14,  103.63,  114.48,
       132.14,  142.77,  169.61,  182.33,  197.83,  209.56,  223.5 ,
       239.35,  252.31,  263.77,  279.97,  297.53,  310.62,  323.38,
       335.92,  365.67])#,  383.93,  402.83,  417.51,  430.48,  440.9 ,
    #    456.7 ,  468.25,  480.64,  492.07,  505.46,  519.24,  536.92,
       #  552.43,  576.9 ,  596.15,  612.13,  632.1 ,  645.22,  664.01,
       #  686.24,  703.4 ,  713.56,  729.68,  740.07,  752.43,  765.83,
       #  777.35,  789.49,  800.04,  810.43,  824.76,  839.67,  850.88,
       #  860.94,  874.11,  888.13,  904.49,  922.57,  935.73,  947.48,
       #  963.51,  973.88,  990.72, 1003.71, 1020.55, 1032.  , 1042.46,
       # 1055.49, 1078.02, 1099.15, 1121.05, 1132.5 , 1161.08, 1172.75,
       # 1188.24, 1201.83, 1230.68, 1243.77, 1255.68, 1280.08, 1294.65,
       # 1322.31, 1347.66, 1375.94, 1385.98, 1399.58, 1411.88, 1430.66,
       # 1447.55, 1458.09, 1468.23, 1478.57, 1493.51, 1503.51, 1520.58,
       # 1535.84, 1548.77])

AP_idx = []
for ti in AP_times:
       AP_idx.append(np.argmin(np.absolute(ts-ti)))
# Dirac où AP dans times series
AP_series = np.zeros(len(ts))
AP_series[AP_idx] = 1

roi = 3
sigs = np.array([HbT[:,roi], HbO[:,roi], HbR[:,roi]])

# sigs = np.array([HbT, HbO, HbR])


## ALIGN DATA
inf = 30
sup = 100
aligned_data_raw = np.zeros((len(sigs), len(AP_idx), inf+sup))

aligned_data_norm = np.zeros((len(sigs), len(AP_idx), inf+sup))
for idx, sig in enumerate(sigs):
  for i in range(len(AP_idx)):
    plage = sig[int(AP_idx[i]-inf):int(AP_idx[i]+sup)]
    aligned_data_raw[idx,i,:] = plage
    aligned_data_norm[idx,i,:] = (plage-np.min(plage))/(np.max(plage)-np.min(plage))

  aligned_data = aligned_data_raw

In [130]:
%matplotlib qt5
# %matplotlib inline

roi = 3
plt.plot(ts, gcamp[:, roi], c='b', label='GCaMP')
plt.plot(ts, HbT[:, roi], c='g', label='HbT')
# plt.plot(ts, HbO[:, roi], c='r', label='HbO')
# plt.plot(ts, HbR[:, roi], c='k', label='HbR')

# plt.plot(ts, gcamp/np.mean(gcamp), c='b', label='GCaMP')
# plt.plot(ts, HbT/np.mean(HbT)+0.5, c='g', label='HbT')

plt.vlines(AP_times, -2, 4, color='grey', linestyles=('--'), label='air puff')

plt.legend()
plt.show()

In [120]:
fig, axs = plt.subplots(3, 2, figsize=(14,7), width_ratios=[5, 4])
sns.set_context('notebook')

titles = ("HbT", "HbO", "HbR")
titles_av = ("HbT average", "HbO  average", "HbR average")
cols = ('midnightblue', 'midnightblue', 'midnightblue')
cmaps = (clm['PuBu_r'], clm['PuBu_r'], clm['PuBu_r'])


for idx, (sig, cmap, title, title_av) in enumerate(zip(aligned_data, cmaps, titles, titles_av)):
  ax = plt.subplot(3, 2, 2*idx+1)
  ax.set_title(title)
  ax.set_xlabel('time relative to airpuff [s]')
  ax.set_ylabel("trial [-]")
  pos = ax.imshow(aligned_data_norm[idx,:,:], origin='lower', extent=[ts[AP_idx[0]-inf]-ts[AP_idx[0]], ts[AP_idx[0]+sup]-ts[AP_idx[0]], 0, len(AP_times)], aspect='auto', cmap=cmap)
  fig.colorbar(pos, ax=ax)

for idx, (sig, col, title, title_av) in enumerate(zip(aligned_data_raw, cols, titles, titles_av)):
    avg_data = np.mean(sig, axis=0)
    std_data = sem(sig, axis=0)

    ax = plt.subplot(3, 2, 2*idx+2)
    ax.set_title(title_av)
    # ax.vlines(0, avg_data.min(), avg_data.max(), color='grey', linestyles='-')
    ax.plot(ts[AP_idx[0]-inf:AP_idx[0]+sup]-ts[AP_idx[0]], avg_data, color=col)
    ax.fill_between(ts[AP_idx[0]-inf:AP_idx[0]+sup]-ts[AP_idx[0]], avg_data-std_data, avg_data+std_data, color=col, alpha=0.2)
    ax.set_xlim(ts[AP_idx[0]-inf]-ts[AP_idx[0]], ts[AP_idx[0]+sup]-ts[AP_idx[0]])
    ax.set_xlabel('time relative to airpuff [s]')
    if idx == 1:
        ax.set_ylabel("Concentration variation [uM]")
    # sns.despine()
    plt.tight_layout()

# plt.savefig("behavior_rasters.svg")

## Corrélation

In [110]:
roi = 0

correlation_HbO = correlate(zscore(gcamp[:,roi]), zscore(HbO[:,roi]))
correlation_HbR = correlate(gcamp[:,roi], -HbR[:,roi])   # anticorrélé
correlation_HbT = correlate(gcamp[:,roi], HbT[:,roi])

# lag_HbO = (len(AP_series) - np.argmax(correlation_HbO))*time[1]
# lag_HbR = (len(AP_series) - np.argmax(correlation_HbR))*time[1]
# lag_HbT = (len(AP_series) - np.argmax(correlation_HbT))*time[1]

plt.plot(correlation_HbO)

In [ ]:
## cropping data

